In [3]:
!pip install -q faiss-cpu rank_bm25 langchain-cohere 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


In [4]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.1/571.1 kB 15.2 MB/s eta 0:00:0000:01
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [3]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# hugging_face_token = user_secrets.get_secret(" HUGGINGFACE_TOKEN")
# cohere_api_key = user_secrets.get_secret("COHERE_API_KEY")
# groq_api_key = user_secrets.get_secret("GROQ_API_KEY")
# wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

In [7]:
# Enable parallel tokenization
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Retrieve and set Cohere API Key
os.environ["COHERE_API_KEY"] = user_secrets.get_secret("COHERE_API_KEY")

# Retrieve and set Hugging face API Key
os.environ[" HUGGINGFACE_TOKEN"] = user_secrets.get_secret(" HUGGINGFACE_TOKEN")

In [6]:
import os
from kaggle_secrets import UserSecretsClient
from openai import OpenAI

# Retrieve the API key from Kaggle secrets
user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")

# Set the environment variable
os.environ["GROQ_API_KEY"] = groq_api_key

# Initialize OpenAI-compatible client for Groq
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# Test request with Groq's Llama 3 model
response = client.chat.completions.create(
    model="llama3-8b-8192",  # Other options: "mixtral-8x7b-32768", "gemma-7b-it"
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is Das Kapital?"}
    ]
)

# Print the response
print(response.choices[0].message.content)

Das Kapital, literally "The Capital" in German, is a foundational work of Marxism written by Karl Marx in 1867. It's a critique of capitalist society and an analysis of the economic systems that existed in the 19th century.

Das Kapital focuses on the concept of surplus value, which Marx believed was the source of profit for capitalist enterprises. According to Marx, workers produce value but are paid less than the value they create, resulting in surplus value that is extracted by the capitalist class. He argued that this exploitation is the fundamental driving force behind the dynamic and constantly changing nature of capitalist societies.

The book is divided into three volumes, each exploring different aspects of capitalism:

1. Das Kapital, Volume I: Marx analyzes the inner workings of capitalist production, discussing issues like the labor theory of value, alienation of labor, and the concept of surplus value.
2. Das Kapital, Volume II: This volume examines the circulation of capi

In [8]:
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain_cohere import CohereRerank
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

In [9]:
import os
dataset_path = "/kaggle/input/das-kapital"
print("Files inside dataset:", os.listdir(dataset_path))

Files inside dataset: ['DasKapital (Volume III).txt', 'DasKapital (Volume II).txt', 'DasKapital (Volume I).txt']


In [10]:
from datasets import Dataset

# Function to read and clean text
def read_and_clean(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return " ".join(text.split())  # Normalize spaces

# Read all text files
file_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(".txt")]
texts = [read_and_clean(f) for f in file_paths]

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({"text": texts})

# Show the first 500 characters
print("Dataset loaded successfully!")
print(dataset[0]["text"][:500])

Dataset loaded successfully!
Capital, Vol.3, Table of Contents Karl Marx's CAPITAL Vol. Ill THE PROCESS OF CAPITALIST PRODUCTION AS A WHOLE Written: Karl Marx, 1863-1883, edited by Friedrick Engels and completed by him eleven years after Marx's death. Source: Institute of Marxism-Leninism, USSR, 1959 Publisher: International Publishers, NY, [n.d.] First Published: 1894 Translated: On-Line Version: Marx.org 1996, Marxists.org 1999 Transcribed: Transcribed for the Internet in 1996 by Hinrich Kuhls and Zodiac, and by Tim Delan


In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the LLM using Groq's OpenAI-compatible API
llm = ChatOpenAI(
    openai_api_key=os.getenv("GROQ_API_KEY"),
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="mixtral-8x7b-32768",  # Other options: "mixtral-8x7b-32768", "gemma-7b-it"
    temperature=0.0,
)

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en",  # Replace with your preferred model
    model_kwargs={"device": "cuda"}  # Use GPU if available
)

### LOAD DATA

In [37]:
WHOLE_DOCUMENT = dataset[0]["text"]

In [38]:
prompt_document = PromptTemplate(
    input_variables=["WHOLE_DOCUMENT"], template="{WHOLE_DOCUMENT}"
)
prompt_chunk = PromptTemplate(
    input_variables=["CHUNK_CONTENT"],
    template="Here is the chunk we want to situate within the whole document\n\n{CHUNK_CONTENT}\n\n"
    "Please give a short succinct context to situate this chunk within the overall document for "
    "the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else.",
)

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import BaseDocumentCompressor
from langchain_core.retrievers import BaseRetriever


def split_text(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=200)
    doc_chunks = text_splitter.create_documents(texts)
    for i, doc in enumerate(doc_chunks):
        # Append a new Document object with the appropriate doc_id
        doc.metadata = {"doc_id": f"doc_{i}"}
    return doc_chunks


def create_embedding_retriever(documents_):
    vector_store = FAISS.from_documents(documents_, embedding=embeddings)
    return vector_store.as_retriever(search_kwargs={"k": 4})


def create_bm25_retriever(documents_):
    retriever = BM25Retriever.from_documents(documents_, language="english")
    return retriever


# Function to create a combined embedding and BM25 retriever with reranker
class EmbeddingBM25RerankerRetriever:
    def __init__(
        self,
        vector_retriever: BaseRetriever,
        bm25_retriever: BaseRetriever,
        reranker: BaseDocumentCompressor,
    ):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        self.reranker = reranker

    def invoke(self, query: str):
        vector_docs = self.vector_retriever.invoke(query)
        bm25_docs = self.bm25_retriever.invoke(query)

        combined_docs = vector_docs + [
            doc for doc in bm25_docs if doc not in vector_docs
        ]

        reranked_docs = self.reranker.compress_documents(combined_docs, query)
        return reranked_docs

In [40]:
chunks = split_text([WHOLE_DOCUMENT])

embedding_retriever = create_embedding_retriever(chunks)

# Define a BM25 retriever
bm25_retriever = create_bm25_retriever(chunks)

reranker = CohereRerank(top_n=3, model="rerank-english-v2.0")

# Create combined retriever
embedding_bm25_retriever_rerank = EmbeddingBM25RerankerRetriever(
    vector_retriever=embedding_retriever,
    bm25_retriever=bm25_retriever,
    reranker=reranker,
)

In [41]:
# import tqdm as tqdm
# from langchain.docstore.document import Document


# def create_contextual_chunks(chunks_):
#     # uses a llm to add context to each chunk given the prompts defined above
#     contextual_documents = []
#     for chunk in tqdm.tqdm(chunks_):
#         context = prompt_document.format(WHOLE_DOCUMENT=WHOLE_DOCUMENT)
#         chunk_context = prompt_chunk.format(CHUNK_CONTENT=chunk)
#         llm_response = llm.invoke(context + chunk_context).content
#         page_content = f"""Text: {chunk.page_content}\n\n\nContext: {llm_response}"""
#         doc = Document(page_content=page_content, metadata=chunk.metadata)
#         contextual_documents.append(doc)
#     return contextual_documents
# contextual_documents = create_contextual_chunks(chunks)

In [43]:
import tqdm
from langchain.docstore.document import Document

# Define a max token limit per request (Keep well below Groq's daily limit)
MAX_TOKENS_PER_CHUNK = 4000  # Adjust as needed

# Function to split long chunks before sending them to the LLM
def split_chunk(chunk, max_tokens=MAX_TOKENS_PER_CHUNK):
    words = chunk.page_content.split()
    sub_chunks = [" ".join(words[i:i + max_tokens]) for i in range(0, len(words), max_tokens)]
    return sub_chunks

# Function to create contextual chunks with optimized size
def create_contextual_chunks(chunks_):
    contextual_documents = []
    
    for chunk in tqdm.tqdm(chunks_):
        sub_chunks = split_chunk(chunk)  # Reduce chunk size before sending
        for sub_chunk in sub_chunks:
            context = prompt_document.format(WHOLE_DOCUMENT=WHOLE_DOCUMENT[:1000])  # Truncate document context
            chunk_context = prompt_chunk.format(CHUNK_CONTENT=sub_chunk)  
            
            # Send smaller request to LLM
            llm_response = llm.invoke(context + chunk_context).content  
            page_content = f"""Text: {sub_chunk}\n\n\nContext: {llm_response}"""
            
            # Store results
            doc = Document(page_content=page_content, metadata=chunk.metadata)
            contextual_documents.append(doc)

    return contextual_documents

# Call the optimized function
contextual_documents = create_contextual_chunks(chunks)

 76%|███████▋  | 470/615 [1:48:55<33:36, 13.90s/it]  


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01jmrvjggffw8vf3vw6zy16n9e` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499143, Requested 1379. Please try again in 1m30.1506s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}